### Task-I: Build and populate necessary tables (30% of course project grade)
1) Ingest both train and test data into one Postgres Database table. Use the augmented datasets that are provided under Final CSV folder.
2) Add a field to your database table that distinguishes between train and test datasets.
3) Identify constraints as needed and document them in your Readme.md file.
4) Your tables should be created in schema with the name “mqtt”.
5) In your ReadMe.md, add a description for the features in the dataset.
6) Use the reduced version of the data if your laptop’s memory can’t handle the original dataset.

In [1]:
# Uncomment the following lines if you are using Windows!
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder.getOrCreate()

C:\spark\spark-3.4.1-bin-hadoop3\python\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


This Schema has been created after manually observing the different values in the .csv file. For Hexadecimal vals, string type is used and for mqtt_msg string type is used.

In [2]:
sqlWay = spark.sql("""
CREATE SCHEMA mqtt;
""")

# sqlWay.show()

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType


mqtt = StructType([
    StructField("tcp_flags", StringType(), True),
    StructField("tcp_time_delta", DoubleType(), True),
    StructField("tcp_len", IntegerType(), True),
    StructField("mqtt_conack_flags", StringType(), True),
    StructField("mqtt_conack_flags_reserved", IntegerType(), True),
    StructField("mqtt_conack_flags_sp", IntegerType(), True),
    StructField("mqtt_conack_val", IntegerType(), True),
    StructField("mqtt_conflag_cleansess", IntegerType(), True),
    StructField("mqtt_conflag_passwd", IntegerType(), True),
    StructField("mqtt_conflag_qos", IntegerType(), True),
    StructField("mqtt_conflag_reserved", IntegerType(), True),
    StructField("mqtt_conflag_retain", IntegerType(), True),
    StructField("mqtt_conflag_uname", IntegerType(), True),
    StructField("mqtt_conflag_willflag", IntegerType(), True),
    StructField("mqtt_conflags", StringType(), True),
    StructField("mqtt_dupflag", IntegerType(), True),
    StructField("mqtt_hdrflags", StringType(), True),
    StructField("mqtt_kalive", IntegerType(), True),
    StructField("mqtt_len", IntegerType(), True),
    StructField("mqtt_msg", StringType(), True),
    StructField("mqtt_msgid", IntegerType(), True),
    StructField("mqtt_msgtype", IntegerType(), True),
    StructField("mqtt_proto_len", IntegerType(), True),
    StructField("mqtt_protoname", StringType(), True),
    StructField("mqtt_qos", IntegerType(), True),
    StructField("mqtt_retain", IntegerType(), True),
    StructField("mqtt_sub_qos", IntegerType(), True),
    StructField("mqtt_suback_qos", IntegerType(), True),
    StructField("mqtt_ver", IntegerType(), True),
    StructField("mqtt_willmsg", IntegerType(), True),
    StructField("mqtt_willmsg_len", IntegerType(), True),
    StructField("mqtt_willtopic", IntegerType(), True),
    StructField("mqtt_willtopic_len", IntegerType(), True),
    StructField("target", StringType(), True)
])

In [3]:
df_train = spark.read.csv("train70_reduced.csv" ,header=True, inferSchema= True)
df_test = spark.read.csv("test30_reduced.csv", header=True, inferSchema=True)

# df_train = spark.read.csv("train70_reduced.csv" ,header=True, inferSchema= False).schema(mqtt)
# df_test = spark.read.csv("test30_reduced.csv", header=True, inferSchema=False).schema(mqtt)

# df_train = spark.read.format('csv').schema(mqtt).option('header', True).load('train70_reduced.csv')
# df_test = spark.read.format('csv').schema(mqtt).option('header', True).load('test30_reduced.csv')

In [4]:
from pyspark.sql.functions import lit

df_train = df_train.withColumn('Train', lit(1))
df_test = df_test.withColumn('Train', lit(0))

In [5]:
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="18763kebjeseaya"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt"


df_train.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

df_test.write.format("jdbc")\
.mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [6]:
df = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp.flags                  | 0x00000018 
 tcp.time_delta             | 0.998867   
 tcp.len                    | 10         
 mqtt.conack.flags          | 0          
 mqtt.conack.flags.reserved | 0.0        
 mqtt.conack.flags.sp       | 0.0        
 mqtt.conack.val            | 0.0        
 mqtt.conflag.cleansess     | 0.0        
 mqtt.conflag.passwd        | 0.0        
 mqtt.conflag.qos           | 0.0        
 mqtt.conflag.reserved      | 0.0        
 mqtt.conflag.retain        | 0.0        
 mqtt.conflag.uname         | 0.0        
 mqtt.conflag.willflag      | 0.0        
 mqtt.conflags              | 0          
 mqtt.dupflag               | 0.0        
 mqtt.hdrflags              | 0x00000030 
 mqtt.kalive                | 0.0        
 mqtt.len                   | 8.0        
 mqtt.msg                   | 32         
 mqtt.msgid                 | 0.0        
 mqtt.msgtype               | 3.0        
 mqtt.proto_len             | 0.0 

In [7]:
cols = df.columns

for column in cols:
    new_column = column.replace('.', '_')
    df = df.withColumnRenamed(column, new_column)
    
df.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

In [8]:
print(df_train.count())
print(df_test.count())
print(f'{df.count()} {df_train.count()+df_test.count()}')

231646
99290
330936 330936


In [9]:
for column in df.columns:
    print(f"{df.agg({column: 'max'}).collect()[0][0]}")

0x00000019
60.000878
32768
0x00000000
0.0
0.0
5.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0x000000c2
1.0
0x000000e0
65535.0
692.0
746573747465737474657374
10000.0
14.0
4.0
MQTT
1.0
1.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
slowite
1


### Task-II: Conduct analytics on your dataset (20% of course project grade)
Develop Python functions that run Spark to answer the following questions. All of the core analysis and data ingestion should be conducted via PySpark. Ingest all the data to answer the following questions from the Postgres Database table.
1. What is the average length of an MQTT message captured in the training dataset?
2. For each target value, what is the average length of the TCP message? (Conduct this process programmatically and don’t hardcode any of the target values in your command)
3. Build a Python function that uses PySpark to list the most frequent X TCP flags where X is a user-provided parameter.
    o Make sure to handle this scenario as well: if the user requests 5 most frequent TCP flags but there are 3 Flags that share the same count at rank number 5, please include all of them in your output.
4. Among the listed targets, what is the most popular target on Google News? (Use 5-minutes Google News feed to justify your answer).
    o Use this query: https://news.google.com/rss/search?q=popular+cyber+attacks
    o You may find yourself in need to decrypt the target values in the dataset to proper English equivalent. For example, “bruteforce” to “brute force”.

In [13]:
# (1)
from pyspark.sql.functions import length, avg
from pyspark.sql.functions import col

df_train = df.where(df.Train == 1)
# print(df_train.count())
df_mqtt_msg_avg = df_train.select(col('mqtt_len')).summary("mean")
df_mqtt_msg_avg.show()



+-------+------------------+
|summary|          mqtt_len|
+-------+------------------+
|   mean|31.435725201384873|
+-------+------------------+



In [14]:
df_mqtt_msg_avg.show()

+-------+------------------+
|summary|          mqtt_len|
+-------+------------------+
|   mean|31.435725201384873|
+-------+------------------+



### Correcting this from the comment received at checkpoint submission

In [15]:
# (2)


df.select('target', 'tcp_len').show(5)

# Correcting this from the comment received at checkpoint submission

df_tcp_msg_avg_len = df.groupBy('target').agg(avg('tcp_len').alias('tcp_msg_avg_len'))
df_tcp_msg_avg_len.show()

+----------+-------+
|    target|tcp_len|
+----------+-------+
|legitimate|     10|
|       dos|   1460|
|       dos|   1460|
|legitimate|     10|
|       dos|     16|
+----------+-------+
only showing top 5 rows

+----------+------------------+
|    target|   tcp_msg_avg_len|
+----------+------------------+
|   slowite|3.9993479678330797|
|bruteforce|3.9871043376318873|
|     flood|13313.415986949429|
| malformed| 20.97491761259612|
|       dos|312.65759830457716|
|legitimate| 7.776101001432345|
+----------+------------------+



In [16]:
# (2)


df.select('target', 'mqtt_len').show(5)

df_tcp_msg_avg_len = df.groupBy('target').agg(avg('mqtt_len')).alias('mqtt_msg_avg_len')
df_tcp_msg_avg_len.show()

+----------+--------+
|    target|mqtt_len|
+----------+--------+
|legitimate|     8.0|
|       dos|   169.0|
|       dos|   163.0|
|legitimate|     8.0|
|       dos|     2.0|
+----------+--------+
only showing top 5 rows

+----------+------------------+
|    target|     avg(mqtt_len)|
+----------+------------------+
|   slowite|3.5331449684851117|
|bruteforce|2.9956554720364115|
|     flood| 7.491027732463295|
| malformed| 6.263181984621018|
|       dos| 71.13934256294488|
|legitimate| 6.515196750935254|
+----------+------------------+



In [17]:
# num_distinct_counts = df_tcp_flags.select('count').count()
# num_distinct_counts

In [18]:
# from pyspark.sql.functions import sum, col, desc
# df_tcp_flags = df.groupBy('tcp_flags').count().orderBy(desc('count'))
# x = 5
# num_to_show = 0
# num_distint = 1

# values_list = df_tcp_flags.select('count').rdd.map(lambda row: row[0]).collect()
# # len(values_list)
# while i < len(values_list):
#     while values_list[i] == values_list[i+1]:
#         i += 1
#     num_distint += 1
    
    
    

# # df_tcp_flags.show()

In [19]:
from pyspark.sql.functions import desc

def get_frequent_tcp_flags(x, df):
    df_temp = df.groupBy('tcp_flags').count().orderBy(desc('count'))
    num_distinct_counts = df_temp.select('count').count()
    if x > num_distinct_counts:
        print('x out of bounds, give a smaller number')
        return
    print(df_temp.show(x))
#     values_list = df_tcp_flags.select('count').rdd.map(lambda row: row[0]).collect()

In [20]:
get_frequent_tcp_flags(8, df)

+----------+------+
| tcp_flags| count|
+----------+------+
|0x00000018|183076|
|0x00000010|134547|
|0x00000011|  4198|
|0x00000002|  3372|
|0x00000012|  3372|
|0x00000004|  1592|
|0x00000019|   738|
|0x00000014|    41|
+----------+------+

None


In [21]:
# from confluent_kafka import Producer
# import socket
# #Initialize Your Parameters here - Keep the variable values as is for the ones you can't find on the Confluent-Kafka connection 
# KAFKA_CONFIG = {
#     "bootstrap.servers":"pkc-lzvrd.us-west4.gcp.confluent.cloud:9092",
#     "security.protocol":"SASL_SSL",
#     "sasl.mechanisms":"PLAIN",
#     "sasl.username":"SNZEP2LF5EDA6R7A",
#     "sasl.password":"kB1YfUEB5YPurgB/Ma451fdyKlE5KNMa7KHtoJ8miKX2/A2qcAnUZLMrSFFAX7NY",
#     "session.timeout.ms":"45000",
#     "group.id":"python-group-1",
#     'auto.offset.reset': 'smallest',
#     'client.id': socket.gethostname()
# }

# # Update your topic name
# topic_name = "topic_0"
# producer = Producer(KAFKA_CONFIG)


From new Kafka account

In [22]:
from confluent_kafka import Producer
import socket
#Initialize Your Parameters here - Keep the variable values as is for the ones you can't find on the Confluent-Kafka connection 
KAFKA_CONFIG = {
    "bootstrap.servers":"pkc-6ojv2.us-west4.gcp.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"W7IXLJ6KRDZDCGIX",
    "sasl.password":"31A2pIPznXDYyKik4ZBMfCIWfr5Jc37LtqCNtRdd3m1faPWZyU7PKKV8vT3EDn8T",
    "session.timeout.ms":"45000",
    "group.id":"python-group-1",
    'auto.offset.reset': 'smallest',
    'client.id': socket.gethostname()
}

# Update your topic name
topic_name = "topic_0"
producer = Producer(KAFKA_CONFIG)


In [23]:
import feedparser
import time

# We are searching for Analytics in the news
feed_url = "https://news.google.com/rss/search?q=popular+cyber+attacks"
duration_minutes = 5


def extract_news_feed(feed_url):
    start_time = time.time()
    end_time = start_time + (duration_minutes * 60)
    
    feed = feedparser.parse(feed_url)
    articles = []
    extracted_articles = set()
    
    seconds = 60
    print(f'Start time: {0} sec...')
    while time.time() < end_time:
        for entry in feed.entries:
            if time.time() >= end_time:
                break
            if time.time()-start_time > seconds:
                print(f'Checkpoint {time.time()-start_time} sec...')
                seconds += 60
            link = entry.link
            title = entry.title.encode('ascii', 'ignore').decode()
            unique_id = f'{link}-{title}'
            if unique_id in extracted_articles:
                continue
            extracted_articles.add(unique_id)
            article_data = {"title": title, "link":link}
            if article_data is not None:
                producer.produce(topic_name, key=article_data["title"], value=article_data["link"])
        producer.flush()
    print(f'End time: {time.time()-start_time} sec...')

    
extract_news_feed(feed_url)

Start time: 0 sec...
Checkpoint 60.000038862228394 sec...
Checkpoint 120.00352311134338 sec...
Checkpoint 180.00857782363892 sec...
Checkpoint 240.0008602142334 sec...
End time: 300.0000092983246 sec...


In [24]:
from confluent_kafka import Consumer
from pyspark.sql.types import *
import string
import time


# Clean the punctation by making a translation table that maps punctations to empty strings
translator = str.maketrans("", "", string.punctuation)


emp_RDD = spark.sparkContext.emptyRDD()
# Defining the schema of the DataFrame
columns = StructType([StructField('key', StringType(), False),
                      StructField('value', StringType(), False)])

# Creating an empty DataFrame
df2 = spark.createDataFrame(data=emp_RDD,
                                   schema=columns)
 
# Printing the DataFrame with no data
df2.show()

consumer = Consumer(KAFKA_CONFIG)
consumer.subscribe([topic_name])

try:
    i = 0
    while i < 5:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            i = i + 1
            print("Waiting...")
            continue
        if msg is not None:
            key = msg.key().decode('utf-8').lower().translate(translator)
            cleaned_key = " ".join(key.split())
            value = msg.value().decode('utf-8')
            added_row = [[cleaned_key,value]]
            added_df2 = spark.createDataFrame(added_row, columns)
            df2 = df2.union(added_df2)



except KeyboardInterrupt:
    pass
finally:
    consumer.close()
    df2.show()


+---+-----+
|key|value|
+---+-----+
+---+-----+

Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|is the fear of cy...|https://news.goog...|
|lockbit hackers p...|https://news.goog...|
|the race to adapt...|https://news.goog...|
|danish critical i...|https://news.goog...|
|cyber security to...|https://news.goog...|
|hackers target gr...|https://news.goog...|
|spooky cyber stat...|https://news.goog...|
|cyber attacks aga...|https://news.goog...|
|how to protect ag...|https://news.goog...|
|boosting cybersec...|https://news.goog...|
|the ico and the n...|https://news.goog...|
|customs collabora...|https://news.goog...|
|pentesting vs pen...|https://news.goog...|
|the 6 most common...|https://news.goog...|
|the top 6 cyber s...|https://news.goog...|
|the alarming cybe...|https://news.goog...|
|director wrays op...|https://news.goog...|
|china ukraine a

In [25]:
## scan full dataset
from pyspark.sql.functions import *
# response = table.scan(AttributesToGet=['protocol_type'])
# items = response['Items']

# while 'LastEvaluatedKey' in response:
#     response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
#     items.extend(response['Items'])
    
# response['Items'] = items
    
# unique_values = set()

# if 'Items' in response:
#     for item in response['Items']:
#         if 'protocol_type' in item:
#             unique_values.add(item['protocol_type'])

attacks = ['slowite', 'brute force', 'flood', 'malformed', 'dos', 'ddos', 'legitimate', 'denial-of-service', 'cyber', 'hack']


streamed_data = df2.withColumn('word', explode(split(col('key'), ' '))) \
                .filter(col('word').isin(attacks)) \
                .groupBy('word') \
                .count() \
                .sort('count', ascending=False)
    
streamed_data.show()

+-----+-----+
| word|count|
+-----+-----+
|cyber|   66|
| hack|    1|
+-----+-----+

